In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from GradientBoostingRegressor import GradientBoostingRegressor

In [2]:
df_train = pd.read_csv('data/regression.train', header=None, sep='\t')
df_test = pd.read_csv('data/regression.test', header=None, sep='\t')
y_train = df_train[0].values
y_test = df_test[0].values
X_train = df_train.drop(0, axis=1).values
X_test = df_test.drop(0, axis=1).values

In [3]:
reg = GradientBoostingRegressor()
reg.fit(X_train, y_train, X_test, y_test)

Iteration: 1  Train Loss: 0.20843002  Val Loss: 0.20741951
Iteration: 2  Train Loss: 0.19658527  Val Loss: 0.20081958
Iteration: 3  Train Loss: 0.19012479  Val Loss: 0.19769189
Iteration: 4  Train Loss: 0.18697714  Val Loss: 0.19393727
Iteration: 5  Train Loss: 0.18452566  Val Loss: 0.19162197
Iteration: 6  Train Loss: 0.18347792  Val Loss: 0.19247375
Iteration: 7  Train Loss: 0.18197433  Val Loss: 0.19145186
Iteration: 8  Train Loss: 0.18278353  Val Loss: 0.19340768
Iteration: 9  Train Loss: 0.18122458  Val Loss: 0.19200595
Iteration: 10  Train Loss: 0.18026935  Val Loss: 0.19159117
Iteration: 11  Train Loss: 0.17983054  Val Loss: 0.19150433
Iteration: 12  Train Loss: 0.17968739  Val Loss: 0.19251485
Early Stopping  Best Iteration: 7  Best Val Loss: 0.191452
Training Finished


In [4]:
y_pred = reg.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

0.4375521171244936

In [5]:
# comparing with LightGBM
# reference: https://github.com/Microsoft/LightGBM/blob/master/examples/python-guide/simple_example.py

In [6]:
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_squared_error

print('Loading data...')
# load or create your dataset
df_train = pd.read_csv('data/regression.train', header=None, sep='\t')
df_test = pd.read_csv('data/regression.test', header=None, sep='\t')

y_train = df_train[0]
y_test = df_test[0]
X_train = df_train.drop(0, axis=1)
X_test = df_test.drop(0, axis=1)

# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

print('Starting predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
# eval
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

Loading data...
Starting training...
[1]	valid_0's l1: 0.492841	valid_0's l2: 0.243898
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l1: 0.489327	valid_0's l2: 0.240605
[3]	valid_0's l1: 0.484931	valid_0's l2: 0.236472
[4]	valid_0's l1: 0.480567	valid_0's l2: 0.232586
[5]	valid_0's l1: 0.475965	valid_0's l2: 0.22865
[6]	valid_0's l1: 0.472861	valid_0's l2: 0.226187
[7]	valid_0's l1: 0.469847	valid_0's l2: 0.223738
[8]	valid_0's l1: 0.466258	valid_0's l2: 0.221012
[9]	valid_0's l1: 0.462751	valid_0's l2: 0.218429
[10]	valid_0's l1: 0.458755	valid_0's l2: 0.215505
[11]	valid_0's l1: 0.455252	valid_0's l2: 0.213027
[12]	valid_0's l1: 0.452051	valid_0's l2: 0.210809
[13]	valid_0's l1: 0.448764	valid_0's l2: 0.208612
[14]	valid_0's l1: 0.446667	valid_0's l2: 0.207468
[15]	valid_0's l1: 0.444211	valid_0's l2: 0.206009
[16]	valid_0's l1: 0.44186	valid_0's l2: 0.20465
[17]	valid_0's l1: 0.438508	valid_0's l2: 0.202489
[18]	valid_0's l1: 0.435919	valid_0's l2: 0.200